#  Simple tool to analyze data from www.data.gouv.fr

**Note:** This is a Jupyter notebook which is also available as its executable export as a Python 3 script (therefore with automatically generated comments).

# Libraries

In [1]:
# Sys import
import sys, os, re
# Common imports
import math
import numpy             as NP
import numpy.random      as RAND
import scipy.stats       as STATS
from scipy import sparse
from scipy import linalg

# Better formatting functions
from IPython.display import display, HTML
from IPython import get_ipython

import matplotlib        as MPL
import matplotlib.pyplot as PLT
import seaborn as SNS
SNS.set(font_scale=1)

# Python programming
from itertools import cycle
from time import time
import datetime

# Using pandas
import pandas as PAN
import xlrd

In [2]:
import warnings
warnings.filterwarnings('ignore')
print("For now, reduce python warnings, I will look into this later")

For now, reduce python warnings, I will look into this later


### Import my own modules
The next cell attempts to give user some information if things improperly setup.
Intended to work both in Jupyter and when executing the Python file directly.

In [3]:
if not get_ipython() is None and os.path.abspath("../source/") not in sys.path:
    sys.path.append(os.path.abspath("../source/"))
try:
    from lib.utilities     import *
    from lib.figureHelpers import *
    from lib.DataMgrJSON   import *
    from lib.DataMgr       import *
    
    import lib.basicDataCTE as DCTE
except Exception as err:
    print("Could not find library 'lib' with contents 'DataGouvFr' ")
    if get_ipython() is None:
        print("Check the PYTHONPATH environment variable which should point to 'source' wich contains 'lib'")
    else:
        print("You are supposed to be running in JupySessions, and '../source/lib' should exist")
    raise err

Loaded lib
For now, reduce python warnings, I will look into this later


## Check environment

It is expected that:
- your working directory is named `JupySessions`, 
- that it has subdirectories 
   - `images/*` where generated images may be stored to avoid overcrowding. 
- At the same level as your working dir there should be directories 
   - `../data` for storing input data and 
   - `../source` for python scripts.
   
My package library is in `../source/lib`, and users running under Python (not in Jupyter) should
set their PYTHONPATH to include "../source" ( *or whatever appropriate* ).

In [4]:
checkSetup(chap="Chap01")
ImgMgr = ImageMgr(chapdir="Chap01")

# Load Data

## Functions

## Load CSV and XLSX data from remote 
The `dataFileVMgr` will manage a cache of data files in `../data`, the data will be downloaded
from www.data.gouv.fr using a request for datasets with badge '`covid-19`' if a more recent
version is present on the remote site. The meta information is stored/cached  in `../data/.data`
as the pickle of a json.

We check what is in the cache/data directory; for each file, we identify the latest version, 
and list this below to make sure. The file name will usually contain a time stamp; this has to do with 
the version management/identification technique used when downloading from www.data.gouv.fr.

For the files used in this notebook, the latest version is used/loaded irrespective of the
timestamp used in the notebook.

In [5]:
dataFileVMgr = manageAndCacheDataFilesFRDG("../data")
dataFileVMgr.getRemoteInfo()
dataFileVMgr.updatePrepare()
dataFileVMgr.cacheUpdate()

Loaded pickle from ../data/.cache.json, loaded 0h 10m 13s ago (7 elts)
Skipping 'Population présente avant et après le début du confinement' fmt:None mod:2020-04-16T12:08:56.978000 org='institut-national-de-la-statistique-et-des-etudes-economiques-insee'


In [6]:
dataFileVMgr.showMetaData()

cache metadata:{   'data:type': 'python/json',
    'options': {   'ApiHeaders': {},
                   'ApiInq': 'datasets',
                   'CacheValidity': 43200,
                   'HttpHDR': 'https://www.data.gouv.fr/api/1',
                   'InqParmsDir': {'badge': 'covid-19', 'page': 0, 'page_size': 30},
                   'cacheFname': '.cache.json',
                   'httpTimeOut': 1,
                   'maxDirSz': 52428800,
                   'maxImportSz': 5242880},
    'pickleTS': 'Pickled at :2020-05-13 22:35:31'}


In [7]:
print("Most recent versions of files in data directory:")
for f in dataFileVMgr.listMostRecent() :
    print(f"\t{f}")

Most recent versions of files in data directory:
	donnees-hospitalieres-classe-age-covid19-2020-05-13-19h00.csv
	donnees-hospitalieres-covid19-2020-05-13-19h00.csv
	donnees-hospitalieres-etablissements-covid19-2020-05-13-19h00.csv
	donnees-hospitalieres-nouveaux-covid19-2020-05-13-19h00.csv
	donnees-tests-covid19-labo-hebdomadaire-2020-05-13-19h00.csv
	donnees-tests-covid19-labo-quotidien-2020-05-13-19h00.csv
	sursaud-covid19-hebdomadaire-2020-05-13-19h00.csv
	sursaud-covid19-quotidien-2020-05-13-19h00-departement.csv
	sursaud-covid19-quotidien-2020-05-13-19h00-france.csv
	sursaud-covid19-quotidien-2020-05-13-19h00-region.csv
	sursaud-covid19-quotidien-2020-05-13-19h00.xlsx


In [8]:
last = lambda x: dataFileVMgr.getRecentVersion(x,default=True)

This ensures we load the most recent version, so that it is not required to update the list 
below. The timestamps shown in the following sequence will be update by the call to `getRecentVersion`.

In [9]:
dailyDepCsv    = last("sursaud-covid19-quotidien-2020-04-11-19h00-departement.csv")
dailyRegionCsv = last("sursaud-covid19-quotidien-2020-04-11-19h00-region.csv")
dailyFranceCsv = last("sursaud-covid19-quotidien-2020-04-12-19h00-france.csv")
dailyXlsx      = last("sursaud-covid19-quotidien-2020-04-12-19h00.xlsx")
weeklyCsv      = last("sursaud-covid19-hebdomadaire-2020-04-08-19h00.csv")

hospAgeCsv     = last("donnees-hospitalieres-classe-age-covid19-2020-04-11-19h00.csv")
hospNouveauCsv = last("donnees-hospitalieres-nouveaux-covid19-2020-04-11-19h00.csv")
hospCsv        = last("donnees-hospitalieres-covid19-2020-04-11-19h00.csv")
hospEtablCsv   = last("donnees-hospitalieres-etablissements-covid19-2020-04-12-19h00.csv")
weeklyLabCsv   = last("donnees-tests-covid19-labo-hebdomadaire-2020-04-16-10h47.csv")
dailyLabCsv    = last("donnees-tests-covid19-labo-quotidien-2020-04-17-19h00.csv")


S1 = set (dataFileVMgr.listMostRecent())
S2 =set((dailyDepCsv,dailyRegionCsv,dailyFranceCsv, dailyXlsx, weeklyCsv, 
         hospAgeCsv, hospNouveauCsv, hospCsv,  hospEtablCsv, weeklyLabCsv, dailyLabCsv  ))
missing = S1. difference(S2)
if len(missing) > 0:
    print (f"Missing comparing with most recent files in ../data:")
for f in missing:
    print(f"\t{f}")
                
metaHebdoCsv = "../data/metadonnee-urgenceshos-sosmedecins-covid19-hebdo.csv" 
metaQuotRegCsv = "../data/metadonnee-urgenceshos-sosmedecin-covid19-quot-reg.csv"
metaQuotFraCsv = "../data/metadonnee-urgenceshos-sosmedecin-covid19-quot-fra.csv" 
metaQuotDepCsv = "../data/metadonnee-urgenceshos-sosmedecins-covid19-quot-dep.csv"
metaQuotCsv = "../data/metadonnee-urgenceshos-sosmedecin-covid19-quot.csv"
        
metaHospservices = "../data/metadonnees-services-hospitaliers-covid19.csv"
metaHospAge      = "../data/metadonnees-donnees-hospitalieres-covid19-classes-age.csv"
metaHospIncid    = "../data/metadonnees-hospit-incid.csv"
metaHospNouveau  = "../data/metadonnees-donnees-hospitalieres-covid19-nouveaux.csv"
metaHosp         = "../data/metadonnees-donnees-hospitalieres-covid19.csv"
metaHospEtabl    = "../data/donnees-hospitalieres-etablissements-covid19-2020-04-11-19h00.csv"

metaAideEntr     = "../data/metadonnees-aides-aux-entreprises.csv"
metaNivExcDC     = "../data/metadonnees-niveaux-exces-mortalite-covid19.csv"
metaDepist       = "../data/metadonnees-tests-depistage-covid19.csv"

metaSexeCsv = "../data/metadonnees-sexe.csv"
metaRegionsCsv="../data/regions-france.csv"
metaTranchesAgeCsv="../data/code-tranches-dage.csv"

In [10]:
fSolDep_csv = "../data/fonds-solidarite-volet-1-departemental.csv"
fSolDep_xls= "../data/fonds-solidarite-volet-1-departemental.xlsx"
fSolRegNaf_csv= "../data/fonds-solidarite-volet-1-regional-naf.csv"
fSolRegNaf_xls= "../data/fonds-solidarite-volet-1-regional-naf.xls"
indicExcesDCStand_csv= "../data/indicateur-niveaux-exces-mortalite-standardise.csv"
indicExcesDCDep_csv= "../data/niveaux-exces-mortalite-covid19-dep.csv"
indicExcesDCReg_csv= "../data/niveaux-exces-mortalite-covid19-reg.csv"
incoherent_hebdo_xls= "../data/sursaud-covid19-hebdomadaire-incoherence-01042020.xlsx"
incoherent_quot_xls= "../data/sursaud-covid19-quotidien-incoherence-01042020.xlsx"

In [11]:
ad  = lambda x: "../data/"+x
S1 = set (map(ad, dataFileVMgr.listMostRecent(nonTS=True)))
S2 =set((metaHebdoCsv, metaQuotRegCsv,  metaQuotFraCsv, metaQuotCsv, metaQuotDepCsv,
         metaHospservices, metaHospAge, metaHospIncid, metaHosp,  metaHospEtabl, metaRegionsCsv, 
         metaTranchesAgeCsv, metaAideEntr,  metaNivExcDC,  metaDepist, metaHospNouveau,
         fSolDep_csv, fSolDep_xls, fSolRegNaf_csv, fSolRegNaf_xls,
         indicExcesDCStand_csv, indicExcesDCDep_csv, indicExcesDCReg_csv,  
         incoherent_hebdo_xls, incoherent_quot_xls))
missing = S1. difference(S2)
if len(missing) > 0:
    print (f"Missing comparing with non timestamped files in ../data:")
for f in missing:
    print(f"\t{f}")

Missing comparing with non timestamped files in ../data:
	../data/fonds-solidarite-volet-1-regional-naf-latest.csv
	../data/.cache.json
	../data/metadonnees-sexe.csv
	../data/pages-consignes-prefectorales-covid19.json
	../data/fonds-solidarite-volet-1-departemental-naf-latest.csv
	../data/depts2018.txt
	../data/pages-consignes-prefectorales-covid19.csv


Now load the stuff


In [12]:
ad  = lambda x: "../data/"+x

data_fSolDep= read_xlsxPandas(fSolDep_xls,error_bad_lines=False,sep=",")
data_fSolRegNaf= read_xlsxPandas(fSolRegNaf_xls,error_bad_lines=False,sep=",")
data_indicExcesDCStand= read_csvPandas(indicExcesDCStand_csv,error_bad_lines=False,sep=";")
data_indicExcesDCDep= read_csvPandas(indicExcesDCDep_csv,error_bad_lines=False,sep=";")
data_indicExcesDCReg= read_csvPandas(indicExcesDCReg_csv,error_bad_lines=False,sep=";")
data_incoherent_hebdo= read_xlsxPandas(incoherent_hebdo_xls,error_bad_lines=False,sep=",")
data_incoherent_quot= read_xlsxPandas(incoherent_quot_xls,error_bad_lines=False,sep=",")

meta_Hebdo       = read_csvPandas(metaHebdoCsv,     clearNaN=True, error_bad_lines=False,sep=";", header=2)
meta_QuotReg     = read_csvPandas(metaQuotRegCsv, clearNaN=True, error_bad_lines=False,sep=";", header=1)
meta_QuotFra     = read_csvPandas(metaQuotFraCsv, clearNaN=True, error_bad_lines=False,sep=";", header=1)
meta_QuotDepCsv   = read_csvPandas(metaQuotDepCsv, clearNaN=True, error_bad_lines=False,sep=";", header=1)
meta_Quot        = read_csvPandas(metaQuotCsv, clearNaN=True, error_bad_lines=False,sep=";", header=1)
meta_HospServices = read_csvPandas(metaHospservices, clearNaN=True, error_bad_lines=False,sep=";")
meta_HospAge      = read_csvPandas(metaHospAge, clearNaN=True, error_bad_lines=False,sep=";")
meta_HospIncid    = read_csvPandas(metaHospIncid, clearNaN=True, error_bad_lines=False,sep=";")
meta_Hosp         = read_csvPandas(metaHosp, clearNaN=True, error_bad_lines=False,sep=";")
meta_HospNouveau  = read_csvPandas(metaHospNouveau, clearNaN=True, error_bad_lines=False,sep=";")

meta_AideEntr    = read_csvPandas(metaAideEntr, clearNaN=True, error_bad_lines=False,sep=",")  
meta_NivExcDC    = read_csvPandas(metaNivExcDC, clearNaN=True, error_bad_lines=False,sep=";")
meta_Depist      = read_csvPandas(metaDepist, clearNaN=True, error_bad_lines=False,sep=";")

meta_Sexe = read_csvPandas(metaSexeCsv, clearNaN=True, error_bad_lines=False,sep=";",header=0)
meta_Regions = read_csvPandas(metaRegionsCsv, clearNaN=True, error_bad_lines=False,sep=",")
meta_Ages    =  read_csvPandas(metaTranchesAgeCsv, clearNaN=True, error_bad_lines=False,sep=";")

## Figure out data characteristics

In [13]:
def showBasics(data,dataName):
    print(f"{dataName:24}\thas shape {data.shape}")

dataListDescr = ((data_fSolDep, "data_fSolDep"),
                 (data_fSolRegNaf, "data_fSolRegNaf"),
                 (data_indicExcesDCStand, "data_indicExcesDCStand"),
                 (data_indicExcesDCDep, "data_indicExcesDCDep"),
                 (data_indicExcesDCReg, "data_indicExcesDCReg"),
                 (data_incoherent_hebdo, "data_incoherent_hebdo"),
                 (data_incoherent_quot, "data_incoherent_quot"),
                 (meta_AideEntr,  "meta_AideEntr"), 
                 (meta_NivExcDC, "meta_NivExcDC"),
                 (meta_Depist,     "meta_Depist"),    
                  (meta_Hebdo,"meta_Hebdo"),
                  (meta_QuotReg,"meta_QuotReg"),
                  (meta_QuotFra,"meta_QuotFra"),
                  (meta_Quot,"meta_Quot"),
                  (meta_QuotDepCsv,"meta_QuotDepCsv"),
                  (meta_HospServices,"meta_HospServices"),
                  (meta_HospAge,"meta_HospAge"),
                  (meta_HospIncid,"meta_HospIncid"),
                  (meta_HospNouveau, "meta_HospNouveau"), 
                  (meta_Hosp,"meta_Hosp"),
                  (meta_Sexe,"meta_Sexe"),
                  (meta_Regions,'meta_Regions'),
                  (meta_Ages,'meta_Ages'))
    
for (dat,name) in dataListDescr:
    showBasics(dat,name)


data_fSolDep            	has shape (1632, 10)
data_fSolRegNaf         	has shape (307, 8)
data_indicExcesDCStand  	has shape (5, 2)
data_indicExcesDCDep    	has shape (800, 4)
data_indicExcesDCReg    	has shape (136, 4)
data_incoherent_hebdo   	has shape (7, 6)
data_incoherent_quot    	has shape (7, 6)
meta_AideEntr           	has shape (8, 5)
meta_NivExcDC           	has shape (4, 4)
meta_Depist             	has shape (10, 5)
meta_Hebdo              	has shape (8, 5)
meta_QuotReg            	has shape (18, 5)
meta_QuotFra            	has shape (18, 5)
meta_Quot               	has shape (18, 5)
meta_QuotDepCsv         	has shape (18, 5)
meta_HospServices       	has shape (3, 5)
meta_HospAge            	has shape (7, 5)
meta_HospIncid          	has shape (6, 5)
meta_HospNouveau        	has shape (6, 5)
meta_Hosp               	has shape (7, 5)
meta_Sexe               	has shape (3, 2)
meta_Regions            	has shape (18, 2)
meta_Ages               	has shape (6, 2)


### Help with meta data
Of course I encountered some surprises, see `checkRepresentedRegions` issue with unknown codes which
did occur in some files!

In [14]:
def checkRepresentedRegions(df,col='reg',**kwOpts):
    "list regions represented in a dataframe, if kwd print=True, will print list of code->string"
    regs = set(df[col])
    if "print" in kwOpts:
        for r in regs:
            extract = meta_Regions[ meta_Regions['code_region'] == r]
            # print (f"r={r}\t{extract}\t{extract.shape}")
            if extract.shape[0] == 0:
                lib = f"**Unknown:{r}**"
            else:
                lib=extract.iloc[0]. at ['nom_region']
            print(f"Region: code={r}\t->{lib}")
    return regs

In [15]:
for (dat,name) in dataListDescr:
    if name[0:5]=="meta_": continue
    print(f"\nDescription of data in '{name}'\n")
    display(dat.describe().transpose())


Description of data in 'data_fSolDep'



,count,mean,std,min,25%,50%,75%,max
nombre_aides,1632.0,619.712010,9.445008e+02,3.0,94.75,282.0,743.0,14252.0
montant_total,1632.0,822859.158701,1.285379e+06,3101.0,124625.75,369271.0,968386.0,19777816.0
reg,1632.0,52.615809,2.820858e+01,1.0,27.00,52.0,76.0,94.0



Description of data in 'data_fSolRegNaf'



,count,mean,std,min,25%,50%,75%,max
nombre_aides,307.0,3.294622e+03,5.000932e+03,3.0,217.5,1123.0,4471.0,28996.0
montant_total,307.0,4.374641e+06,6.762688e+06,3310.0,295122.5,1498450.0,5698493.5,39929847.0
reg,307.0,4.029967e+01,3.223861e+01,1.0,6.0,32.0,75.0,94.0



Description of data in 'data_indicExcesDCStand'



...

,count,mean,std,min,25%,50%,75%,max
reg,136.0,41.352941,32.280146,1.0,11.0,32.0,75.0,94.0
cat_zscore,136.0,1.764706,1.248616,1.0,1.0,1.0,2.0,5.0



Description of data in 'data_incoherent_hebdo'



,count,mean,std,min,25%,50%,75%,max
Nombre de lignes,7.0,88.571429,148.014478,1.000000,10.500000,26.000000,78.000000,416.000000
% de Lignes,7.0,21.291209,35.580403,0.240385,2.524038,6.250000,18.750000,100.000000
Moyenne de la différence,5.0,3.633683,4.873706,1.000000,1.000000,1.269231,2.631579,12.267606
Min de la différence,5.0,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
Max de la différence,5.0,38.000000,75.043321,1.000000,1.000000,4.000000,12.000000,172.000000



Description of data in 'data_incoherent_quot'



,count,mean,std,min,25%,50%,75%,max
Nombre de lignes,7.0,3307.857143,8429.504759,1.00000,13.500000,150.000000,277.500000,22422.000000
% de Lignes,7.0,14.752730,37.594794,0.00446,0.060209,0.668986,1.237624,100.000000
Moyenne de la différence,5.0,1.701557,1.201922,1.00000,1.000000,1.166667,1.523810,3.817308
Min de la différence,5.0,1.000000,0.000000,1.00000,1.000000,1.000000,1.000000,1.000000
Max de la différence,5.0,8.200000,11.882761,1.00000,1.000000,3.000000,7.000000,29.000000


In [16]:
for (dat,name) in dataListDescr:
    if name[0:5]!="meta_": continue
    print(f"\nMeta data in '{name}'\n")
    display(dat)


Meta data in 'meta_AideEntr'



,Colonne,Type,Description_FR,Description_EN,Exemple
0,dispositif,string,nom du fonds concerné,main fund name,Fonds de solidarité
1,volet,string,numéro de volet du fonds de solidarité,funding component ID number,VOLET1
2,nombre_aides,integer,nombre d�aides attribu�es,number of assigned fundings,1397
3,montant_total,integer,montant total des aides attribu�es (en euros),total amount of assigned fundings (in euros),21191.00
4,reg,string,numéro du département géographique,geographical sub-region ID number,84
5,libelle_region,string,nom de la région,geographical region name,Auvergne-Rhône-Alpes
6,code_section,string,Code de la section NAF (secteur d’activité),NAF code section (business sector),S
7,libelle_section,string,Nom de la section NAF correspondante (secteur ...,NAF section name (business sector),Autres activités de services



Meta data in 'meta_NivExcDC'



,Colonne,Description_FR,Description_EN,Exemple
0,dep,Département,Department,44
1,an_sem,Année / Semaine,Year / Week,2020-S11
2,clage_covid_dc65,Classe d'âge,Age group,0 (tous les âges) / A (+65 ans)
3,cat_zscore,Indicateur niveau excès mortalité,Excess mortality indicator,1



Meta data in 'meta_Depist'



...

,Code,Sexe
0,0,femmes + hommes
1,1,hommes
2,2,femmes



Meta data in 'meta_Regions'



,code_region,nom_region
0,1,Guadeloupe
1,2,Martinique
2,3,Guyane
3,4,La Réunion
4,6,Mayotte
5,11,Île-de-France
6,24,Centre-Val de Loire
7,27,Bourgogne-Franche-Comté
8,28,Normandie
9,32,Hauts-de-France



Meta data in 'meta_Ages'



,Code tranches d'age,Unnamed: 1
0,0,tous âges
1,A,moins de 15 ans
2,B,15-44 ans
3,C,45-64 ans
4,D,65-74 ans
5,E,75 et plus


## Get some demographics data from INSEE
For the time being, these data are obtained / loaded from Insee web site using a manual process and are placed in a different directory, therefore a distinct FileManager is used, and loading this data is done here; for more details see the notebook `Pop-Data-FromGouv.ipy`

Using the base version which does not try to update the "../dataPop" directory

In [17]:
dataFileVMgrInsee = manageDataFileVersions("../dataPop") 
inseeDepXLS           ="../dataPop/InseeDep.xls"
inseeDep            = read_xlsxPandas(inseeDepXLS, error_bad_lines=False,sep=",", sheet_name=1, header=7)
inseeReg            = read_xlsxPandas(inseeDepXLS, error_bad_lines=False,sep=",", sheet_name=0, header=7)

Now we can display our demographics data (summarized)

In [18]:
display(inseeDep.iloc[:,4:].sum())
display(inseeReg.iloc[:,4:].sum())

Nombre d'arrondissements         332.0
Nombre de cantons               2041.0
Nombre de communes             34953.0
Population municipale       66524339.0
Population totale           67761092.0
dtype: float64

Nombre de communes          34953
Population municipale    66524339
Population totale        67761092
dtype: int64

# Look at the newer tables

In [19]:
display(data_fSolDep.info())
display(data_fSolDep.describe())
display(data_fSolDep[:10])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1632 entries, 0 to 1631
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dispositif           1632 non-null   object
 1   volet                1632 non-null   object
 2   nombre_aides         1632 non-null   int64 
 3   montant_total        1632 non-null   int64 
 4   reg                  1632 non-null   int64 
 5   libelle_region       1632 non-null   object
 6   dep                  1632 non-null   object
 7   libelle_departement  1632 non-null   object
 8   code_section         1632 non-null   object
 9   libelle_section      1632 non-null   object
dtypes: int64(3), object(7)
memory usage: 127.6+ KB


None

,nombre_aides,montant_total,reg
count,1632.000000,1.632000e+03,1632.000000
mean,619.712010,8.228592e+05,52.615809
std,944.500752,1.285379e+06,28.208583
min,3.000000,3.101000e+03,1.000000
25%,94.750000,1.246258e+05,27.000000
50%,282.000000,3.692710e+05,52.000000
75%,743.000000,9.683860e+05,76.000000
max,14252.000000,1.977782e+07,94.000000


,dispositif,volet,nombre_aides,montant_total,reg,libelle_region,dep,libelle_departement,code_section,libelle_section
0,Fonds de solidarité,VOLET1,132,180207,84,Auvergne-Rhône-Alpes,01,Ain,A,"Agriculture, sylviculture et pêche"
1,Fonds de solidarité,VOLET1,340,442428,84,Auvergne-Rhône-Alpes,01,Ain,C,Industrie manufacturière
2,Fonds de solidarité,VOLET1,22,28350,84,Auvergne-Rhône-Alpes,01,Ain,E,Production et distribution d'eau ; assainissem...
3,Fonds de solidarité,VOLET1,1196,1732876,84,Auvergne-Rhône-Alpes,01,Ain,F,Construction
4,Fonds de solidarité,VOLET1,1367,1856781,84,Auvergne-Rhône-Alpes,01,Ain,G,Commerce ; réparation d'automobiles et de moto...
5,Fonds de solidarité,VOLET1,163,227642,84,Auvergne-Rhône-Alpes,01,Ain,H,Transports et entreposage
6,Fonds de solidarité,VOLET1,1058,1531820,84,Auvergne-Rhône-Alpes,01,Ain,I,Hébergement et restauration
7,Fonds de solidarité,VOLET1,99,128585,84,Auvergne-Rhône-Alpes,01,Ain,J,Information et communication
8,Fonds de solidarité,VOLET1,55,77659,84,Auvergne-Rhône-Alpes,01,Ain,K,Activités financières et d'assurance
9,Fonds de solidarité,VOLET1,176,242396,84,Auvergne-Rhône-Alpes,01,Ain,L,Activités immobilières


In [20]:
display(data_fSolRegNaf.info())
display(data_fSolRegNaf.describe())
display(data_fSolRegNaf[:10])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307 entries, 0 to 306
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   dispositif       307 non-null    object
 1   volet            307 non-null    object
 2   nombre_aides     307 non-null    int64 
 3   montant_total    307 non-null    int64 
 4   reg              307 non-null    int64 
 5   libelle_region   307 non-null    object
 6   code_section     307 non-null    object
 7   libelle_section  307 non-null    object
dtypes: int64(3), object(5)
memory usage: 19.3+ KB


None

,nombre_aides,montant_total,reg
count,307.000000,3.070000e+02,307.000000
mean,3294.622150,4.374641e+06,40.299674
std,5000.931587,6.762688e+06,32.238614
min,3.000000,3.310000e+03,1.000000
25%,217.500000,2.951225e+05,6.000000
50%,1123.000000,1.498450e+06,32.000000
75%,4471.000000,5.698494e+06,75.000000
max,28996.000000,3.992985e+07,94.000000


,dispositif,volet,nombre_aides,montant_total,reg,libelle_region,code_section,libelle_section
0,Fonds de solidarité,VOLET1,2273,3074309,84,Auvergne-Rhône-Alpes,A,"Agriculture, sylviculture et pêche"
1,Fonds de solidarité,VOLET1,5566,7014821,84,Auvergne-Rhône-Alpes,C,Industrie manufacturière
2,Fonds de solidarité,VOLET1,29,38204,84,Auvergne-Rhône-Alpes,D,"Production et distribution d'électricité, de g..."
3,Fonds de solidarité,VOLET1,249,304387,84,Auvergne-Rhône-Alpes,E,Production et distribution d'eau ; assainissem...
4,Fonds de solidarité,VOLET1,16070,23163406,84,Auvergne-Rhône-Alpes,F,Construction
5,Fonds de solidarité,VOLET1,20712,27668860,84,Auvergne-Rhône-Alpes,G,Commerce ; réparation d'automobiles et de moto...
6,Fonds de solidarité,VOLET1,3960,5482733,84,Auvergne-Rhône-Alpes,H,Transports et entreposage
7,Fonds de solidarité,VOLET1,16817,24169016,84,Auvergne-Rhône-Alpes,I,Hébergement et restauration
8,Fonds de solidarité,VOLET1,2007,2616655,84,Auvergne-Rhône-Alpes,J,Information et communication
9,Fonds de solidarité,VOLET1,950,1338528,84,Auvergne-Rhône-Alpes,K,Activités financières et d'assurance


In [21]:
meta_NivExcDC

,Colonne,Description_FR,Description_EN,Exemple
0,dep,Département,Department,44
1,an_sem,Année / Semaine,Year / Week,2020-S11
2,clage_covid_dc65,Classe d'âge,Age group,0 (tous les âges) / A (+65 ans)
3,cat_zscore,Indicateur niveau excès mortalité,Excess mortality indicator,1


In [22]:
display( data_indicExcesDCStand.info())
display( data_indicExcesDCStand.describe())
display( data_indicExcesDCStand[:10])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column                                    Non-Null Count  Dtype 
---  ------                                    --------------  ----- 
 0   Indicateur                                5 non-null      int64 
 1   Niveaux d'excès de mortalité standardisé  5 non-null      object
dtypes: int64(1), object(1)
memory usage: 208.0+ bytes


None

,Indicateur
count,5.000000
mean,3.000000
std,1.581139
min,1.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,5.000000


,Indicateur,Niveaux d'excès de mortalité standardisé
0,1,Pas d'excès
1,2,Excès modéré
2,3,Excès élevé
3,4,Excès très élevé
4,5,Excès exceptionnel


In [23]:
display(data_indicExcesDCDep.info())
display(data_indicExcesDCDep.describe())
display(data_indicExcesDCDep[:10])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   dep               800 non-null    object
 1   an_sem            800 non-null    object
 2   clage_covid_dc65  800 non-null    object
 3   cat_zscore        800 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 25.1+ KB


None

,cat_zscore
count,800.000000
mean,1.496250
std,0.989297
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,5.000000


,dep,an_sem,clage_covid_dc65,cat_zscore
0,2A,2020-S11,A,2
1,2A,2020-S12,A,2
2,2A,2020-S13,A,1
3,2A,2020-S14,A,2
4,2A,2020-S11,0,1
5,2A,2020-S12,0,2
6,2A,2020-S13,0,1
7,2A,2020-S14,0,2
8,2B,2020-S11,A,1
9,2B,2020-S12,A,1


In [24]:
display(data_indicExcesDCReg.info())
display(data_indicExcesDCReg.describe())
display(data_indicExcesDCReg[:10])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   reg               136 non-null    int64 
 1   an_sem            136 non-null    object
 2   clage_covid_dc65  136 non-null    object
 3   cat_zscore        136 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 4.4+ KB


None

,reg,cat_zscore
count,136.000000,136.000000
mean,41.352941,1.764706
std,32.280146,1.248616
min,1.000000,1.000000
25%,11.000000,1.000000
50%,32.000000,1.000000
75%,75.000000,2.000000
max,94.000000,5.000000


,reg,an_sem,clage_covid_dc65,cat_zscore
0,44,2020-S11,A,2
1,44,2020-S12,A,4
2,44,2020-S13,A,5
3,44,2020-S14,A,5
4,44,2020-S11,0,2
5,44,2020-S12,0,5
6,44,2020-S13,0,5
7,44,2020-S14,0,5
8,75,2020-S11,A,1
9,75,2020-S12,A,1


In [25]:
display( data_incoherent_hebdo.info())
display( data_incoherent_hebdo.describe())
display( data_incoherent_hebdo[:10])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Metrique                  7 non-null      object 
 1   Nombre de lignes          7 non-null      int64  
 2   % de Lignes               7 non-null      float64
 3   Moyenne de la différence  5 non-null      float64
 4   Min de la différence      5 non-null      float64
 5   Max de la différence      5 non-null      float64
dtypes: float64(4), int64(1), object(1)
memory usage: 464.0+ bytes


None

,Nombre de lignes,% de Lignes,Moyenne de la différence,Min de la différence,Max de la différence
count,7.000000,7.000000,5.000000,5.0,5.000000
mean,88.571429,21.291209,3.633683,1.0,38.000000
std,148.014478,35.580403,4.873706,0.0,75.043321
min,1.000000,0.240385,1.000000,1.0,1.000000
25%,10.500000,2.524038,1.000000,1.0,1.000000
50%,26.000000,6.250000,1.269231,1.0,4.000000
75%,78.000000,18.750000,2.631579,1.0,12.000000
max,416.000000,100.000000,12.267606,1.0,172.000000


,Metrique,Nombre de lignes,% de Lignes,Moyenne de la différence,Min de la différence,Max de la différence
0,Nombre total de lignes,416,100.000000,NaN,NaN,NaN
1,Lignes avec une erreur de cohérence,85,20.432692,NaN,NaN,NaN
2,Lignes avec une erreur de cohérence pour la va...,2,0.480769,1.000000,1.0,1.0
3,Lignes avec une erreur de cohérence pour la va...,26,6.250000,1.269231,1.0,4.0
4,Lignes avec une erreur de cohérence pour la va...,1,0.240385,1.000000,1.0,1.0
5,Lignes avec une erreur de cohérence pour la va...,19,4.567308,2.631579,1.0,12.0
6,Lignes avec une erreur de cohérence pour la va...,71,17.067308,12.267606,1.0,172.0


In [26]:
display( data_incoherent_quot.info()) 
display( data_incoherent_quot.describe()) 
display( data_incoherent_quot[:10]) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Metrique                  7 non-null      object 
 1   Nombre de lignes          7 non-null      int64  
 2   % de Lignes               7 non-null      float64
 3   Moyenne de la différence  5 non-null      float64
 4   Min de la différence      5 non-null      float64
 5   Max de la différence      5 non-null      float64
dtypes: float64(4), int64(1), object(1)
memory usage: 464.0+ bytes


None

,Nombre de lignes,% de Lignes,Moyenne de la différence,Min de la différence,Max de la différence
count,7.000000,7.000000,5.000000,5.0,5.000000
mean,3307.857143,14.752730,1.701557,1.0,8.200000
std,8429.504759,37.594794,1.201922,0.0,11.882761
min,1.000000,0.004460,1.000000,1.0,1.000000
25%,13.500000,0.060209,1.000000,1.0,1.000000
50%,150.000000,0.668986,1.166667,1.0,3.000000
75%,277.500000,1.237624,1.523810,1.0,7.000000
max,22422.000000,100.000000,3.817308,1.0,29.000000


,Metrique,Nombre de lignes,% de Lignes,Moyenne de la différence,Min de la différence,Max de la différence
0,Nombre total de lignes,22422,100.000000,NaN,NaN,NaN
1,Lignes avec une erreur de cohérence,347,1.547587,NaN,NaN,NaN
2,Lignes avec une erreur de cohérence pour la va...,6,0.026759,1.000000,1.0,1.0
3,Lignes avec une erreur de cohérence pour la va...,150,0.668986,1.166667,1.0,3.0
4,Lignes avec une erreur de cohérence pour la va...,1,0.004460,1.000000,1.0,1.0
5,Lignes avec une erreur de cohérence pour la va...,21,0.093658,1.523810,1.0,7.0
6,Lignes avec une erreur de cohérence pour la va...,208,0.927660,3.817308,1.0,29.0
